In [ ]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
import os

# Optional: Set API Key if not in ~/.autogen/config.json
# os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# Step 1: Define all agent roles with LLM-backed system prompts
admin = AssistantAgent(
    name="AdminAgent",
    system_message="""You are the Admin agent responsible for coordinating the EDA process.
    Distribute the task to other agents (DataPreparerAgent, EDAAgent, VisualizerAgent, etc.)
    and ensure a comprehensive EDA report is generated from a dataset."""
)

data_preparer = AssistantAgent(
    name="DataPreparerAgent",
    system_message="""You are responsible for loading and cleaning the dataset.
    Handle missing values, fix datatypes, and return a clean DataFrame in pandas."""
)

eda_agent = AssistantAgent(
    name="EDAAgent",
    system_message="""You perform exploratory data analysis on the cleaned dataset.
    Return summary statistics, insights, and note interesting patterns."""
)

visualizer = AssistantAgent(
    name="VisualizerAgent",
    system_message="""You generate visualizations like histograms, count plots, and correlation heatmaps
    using matplotlib and seaborn. Save plots to 'eda_outputs/' directory."""
)

critic = AssistantAgent(
    name="CriticAgent",
    system_message="""You review EDA results and visualizations, and provide concise feedback
    about quality, completeness, and clarity of insights."""
)

report_writer = AssistantAgent(
    name="ReportWriterAgent",
    system_message="""You compile all insights, visualizations, and feedback into a markdown report
    named 'eda_report.md' saved under 'eda_outputs/'. Format sections clearly."""
)

executor = AssistantAgent(
    name="ExecutorAgent",
    system_message="""You verify and execute Python code provided by other agents using pandas, matplotlib, seaborn.
    Return error traces if any."""
)

user_proxy = UserProxyAgent(
    name="User",
    human_input_mode="NEVER",  # for automation
    system_message="You represent the data analyst initiating the EDA process."
)

# Step 2: Create group chat
group_chat = GroupChat(
    agents=[
        admin,
        data_preparer,
        eda_agent,
        visualizer,
        critic,
        report_writer,
        executor,
        user_proxy
    ],
    messages=[]
)

# Step 3: Assign group chat manager
manager = GroupChatManager(
    groupchat=group_chat,
    admin_name="AdminAgent"
)

# Step 4: Simulate a user initiating the EDA process
user_proxy.initiate_chat(
    manager,
    message="""
Please perform an end-to-end EDA on the 'titanic.csv' dataset.
- Clean the data
- Generate summary stats and insights
- Create relevant visualizations
- Review results and write a full markdown report
The dataset is available at: https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
"""
)
